In [33]:
from pyspark.sql import SparkSession

In [34]:
spar = SparkSession.builder.appName("examen_opi").getOrCreate()

In [35]:
%%time
df = spark.read.csv("hdfs:///data_opi/all_data.csv",header=True,inferSchema=True)

CPU times: user 30.5 ms, sys: 13.7 ms, total: 44.2 ms
Wall time: 3min


# ¿Cuantos registros hay?

In [15]:
df.count()

62530715

In [16]:
df.printSchema()

root
 |-- producto: string (nullable = true)
 |-- presentacion: string (nullable = true)
 |-- marca: string (nullable = true)
 |-- categoria: string (nullable = true)
 |-- catalogo: string (nullable = true)
 |-- precio: double (nullable = true)
 |-- fechaRegistro: timestamp (nullable = true)
 |-- cadenaComercial: string (nullable = true)
 |-- giro: string (nullable = true)
 |-- nombreComercial: string (nullable = true)
 |-- direccion: string (nullable = true)
 |-- estado: string (nullable = true)
 |-- municipio: string (nullable = true)
 |-- latitud: string (nullable = true)
 |-- longitud: string (nullable = true)



# ¿Cuántas categorías?

In [36]:
df.createOrReplaceTempView("base")

In [37]:
%%time
spark.sql("SELECT COUNT(DISTINCT( categoria )) CountCategoria FROM base ").show()

+--------------+
|CountCategoria|
+--------------+
|            41|
+--------------+

CPU times: user 17 ms, sys: 2.13 ms, total: 19.2 ms
Wall time: 1min 10s


# ¿Cuántas cadenas comerciales están siendo monitoreadas (y , por lo tantos , reportadas en esa base de datos) ?

In [22]:
%%time
spark.sql("SELECT COUNT(DISTINCT( cadenaComercial )) CountCadena FROM base ").show()

+-----------+
|CountCadena|
+-----------+
|        705|
+-----------+

CPU times: user 21.5 ms, sys: 1.89 ms, total: 23.4 ms
Wall time: 1min 27s


# ¿Cuáles son los productos más monitoreados en cada entidad?

In [38]:
%%time
spark.sql("SELECT estado,producto, COUNT(*) numproducto FROM base GROUP BY estado,producto ORDER BY numproducto DESC").show()

+----------------+--------------------+-----------+
|          estado|            producto|numproducto|
+----------------+--------------------+-----------+
|DISTRITO FEDERAL|            REFRESCO|     287463|
|DISTRITO FEDERAL|                 FUD|     207569|
|          MÉXICO|            REFRESCO|     194939|
|DISTRITO FEDERAL|LECHE ULTRAPASTEU...|     175640|
|DISTRITO FEDERAL|   DETERGENTE P/ROPA|     173452|
|          MÉXICO|                 FUD|     149141|
|DISTRITO FEDERAL|             YOGHURT|     136720|
|DISTRITO FEDERAL|             CERVEZA|     136686|
|          MÉXICO|   DETERGENTE P/ROPA|     132862|
|DISTRITO FEDERAL|            MAYONESA|     131103|
|DISTRITO FEDERAL|      CHILES EN LATA|     130598|
|DISTRITO FEDERAL|    JABON DE TOCADOR|     129889|
|DISTRITO FEDERAL|             SHAMPOO|     125603|
|DISTRITO FEDERAL|      LECHE EN POLVO|     116827|
|          MÉXICO|LECHE ULTRAPASTEU...|     116522|
|DISTRITO FEDERAL|         DESODORANTE|     113779|
|DISTRITO FE

# ¿Cuál es la cadena comercial con mayor variedad de productos monitoreados?

In [42]:
%%time
spark.sql("SELECT cadenaComercial,COUNT(DISTINCT( producto )) CountProduct FROM base GROUP BY cadenaComercial ORDER BY CountProduct DESC").show()

+--------------------+------------+
|     cadenaComercial|CountProduct|
+--------------------+------------+
|             SORIANA|        1059|
|            WAL-MART|        1051|
|MEGA COMERCIAL ME...|        1049|
|  COMERCIAL MEXICANA|        1036|
|            CHEDRAUI|        1026|
|     MERCADO SORIANA|        1024|
|      BODEGA AURRERA|        1012|
|HIPERMERCADO SORIANA|        1006|
|              H.E.B.|        1001|
|        SORIANA PLUS|         999|
|       SORIANA SUPER|         996|
|BODEGA COMERCIAL ...|         979|
|        I.S.S.S.T.E.|         937|
|            SUPERAMA|         936|
|              S MART|         851|
|SUPERMERCADOS SAN...|         849|
|              SUMESA|         848|
|         CITY MARKET|         844|
|FARMACIA GUADALAJARA|         819|
|            CASA LEY|         808|
+--------------------+------------+
only showing top 20 rows

CPU times: user 14.7 ms, sys: 11.1 ms, total: 25.8 ms
Wall time: 1min 32s


# Genera una canasta de productos básicos que te permita comparar los precios

In [56]:
%%time
spark.sql("""SELECT producto,cadenaComercial,precio FROM base WHERE producto=="ARROZ"  OR producto=="FRIJOL" ORDER BY producto DESC """).show()

+--------+------------------+------+
|producto|   cadenaComercial|precio|
+--------+------------------+------+
|  FRIJOL|      I.S.S.S.T.E.|  23.6|
|  FRIJOL|      I.S.S.S.T.E.|  21.3|
|  FRIJOL|          WAL-MART|  18.0|
|  FRIJOL|COMERCIAL MEXICANA|  17.0|
|  FRIJOL|          WAL-MART|  22.0|
|  FRIJOL|COMERCIAL MEXICANA|  19.9|
|  FRIJOL|          WAL-MART|  28.0|
|  FRIJOL|COMERCIAL MEXICANA|  15.9|
|  FRIJOL|          WAL-MART|  22.0|
|  FRIJOL|COMERCIAL MEXICANA| 18.15|
|  FRIJOL|          WAL-MART|  24.0|
|  FRIJOL|          WAL-MART|  18.5|
|  FRIJOL|      I.S.S.S.T.E.|  23.6|
|  FRIJOL|          WAL-MART|  28.0|
|  FRIJOL|COMERCIAL MEXICANA| 19.39|
|  FRIJOL|          WAL-MART|  26.0|
|  FRIJOL|COMERCIAL MEXICANA|  15.9|
|  FRIJOL|           SORIANA| 17.85|
|  FRIJOL|COMERCIAL MEXICANA| 20.89|
|  FRIJOL|           SORIANA|  13.7|
+--------+------------------+------+
only showing top 20 rows

CPU times: user 15.1 ms, sys: 8.51 ms, total: 23.7 ms
Wall time: 1min 27s


In [57]:
%%time
spark.sql("""SELECT MAX( precio) FROM base """).show()

+-----------+
|max(precio)|
+-----------+
|   299999.0|
+-----------+

CPU times: user 17.9 ms, sys: 3.66 ms, total: 21.5 ms
Wall time: 1min 19s


# ¿Cuál es la ciudad más cara del país? ¿Cuál es la más barata?

In [73]:
%%time
spark.sql("""SELECT municipio, SUM(precio)/COUNT(nombreComercial) AS Sum_basicos FROM base 
WHERE producto=="ARROZ" OR producto=="FRIJOL" AND cadenaComercial=="WAL-MART" GROUP BY municipio ORDER BY Sum_basicos DESC """).show()

+--------------------+------------------+
|           municipio|       Sum_basicos|
+--------------------+------------------+
|CUAUTITLAN IZCALL...|21.383329369797856|
|GUASAVE          ...|20.793244128891313|
|METEPEC          ...|20.537550607287436|
|             URUAPAN|20.402121212121212|
|COLIMA           ...|19.212859848484847|
|AZCAPOTZALCO     ...| 18.86116093672147|
|GUSTAVO A. MADERO...|18.780478486646878|
|TLALNEPANTLA     ...|18.568545232273834|
|GUADALUPE        ...| 18.56171002295332|
|COACALCO         ...| 18.50836870830806|
|CULIACAN         ...|18.496168183788463|
|VENUSTIANO CARRAN...|18.470665314401625|
|DURANGO          ...| 18.46493455497382|
|          CP. 27280"|18.411999999999992|
|            CULIACÁN|18.265685185185188|
|             GUASAVE|18.240302114803633|
|ZACATECAS        ...|18.130383108402274|
|              XALAPA|18.124552845528456|
|NAUCALPAN        ...| 18.11816225165563|
|PUERTO VALLARTA  ...|18.090298939247827|
+--------------------+------------

In [71]:
%%time
spark.sql("""SELECT municipio, SUM(precio)/COUNT(nombreComercial) AS Sum_basicos FROM base WHERE producto=="ARROZ" OR producto=="FRIJOL" AND cadenaComercial=="WAL-MART" GROUP BY municipio ORDER BY Sum_basicos ASC""").show()

+--------------------+------------------+
|           municipio|       Sum_basicos|
+--------------------+------------------+
|ATIZAPAN         ...|14.309954154727798|
|CARDENAS         ...|14.311086956521738|
|VERACRUZ         ...|14.419512578616354|
|            VERACRUZ|14.508336079077429|
|     OTHÓN P. BLANCO|14.550093632958804|
|TUXTLA GUTIERREZ ...|14.715588235294117|
|OAXACA DE JUAREZ ...| 14.73301503759399|
|OTHON P. BLANCO  ...|14.746417910447764|
|            CÁRDENAS|14.747211155378483|
|    TUXTLA GUTIÉRREZ|14.783108651911471|
|TULTITLAN        ...|14.795691699604742|
|                null|14.908219178082188|
|CAMPECHE         ...|14.940149892933622|
|CARMEN           ...| 14.94031553398058|
|TAPACHULA        ...|14.940694789081888|
|TONALA           ...| 14.94088633993743|
|            ATIZAPÁN|15.016363636363632|
|ATIZAPÁN DE ZARAGOZA|15.024999999999999|
|           TAPACHULA| 15.13490045941807|
|       GÓMEZ PALACIO|15.135941176470594|
+--------------------+------------